In [1]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

# Vetor Data Tutorial

Resources
* https://openmaptiles.org/
* https://github.com/CrunchyData/pg_tileserv
* GeoServer

## Actinia Routes

* (GET, PUT, DELETE) /locations/{location_name}/mapsets/{mapset_name}/vector_layers 
* (GET, POST, DELETE) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}
* (GET) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}/render
* (POST) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}/sampling_async
* (POST) /locations/{location_name}/mapsets/{mapset_name}/vector_layers/{vector_name}/sampling_sync

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/basin_test/vector_layers" | jq

## View Vector Data

In [4]:
!ace location="nc_spm_08" mapset="basin_test" -v

['point_basin']


In [84]:
pc = !ace location="nc_spm_08" mapset="basin_test" script="./scripts/basin.sh" -d | jq -r -c
PC_BODY = pc[0]

In [85]:
str(PC_BODY)

'{"version":"1","list":[{"module":"g.region","id":"g.region_1804289383","inputs":[{"param":"n","value":"252702.4094174725"},{"param":"s","value":"229056.70015573464"},{"param":"e","value":"618028.8126522538"},{"param":"w","value":"598579.6858165322"},{"param":"align","value":"direction_3k_10m_d@basin_test"}]},{"module":"r.circle","id":"r.circle_1804289383","flags":"b","inputs":[{"param":"coordinates","value":"601225.5895524962,241014.7657618352"},{"param":"max","value":"200"}],"outputs":[{"param":"output","value":"circle"}]},{"module":"r.stream.basins","id":"r.stream.basins_1804289383","flags":"c","inputs":[{"param":"direction","value":"direction_3k_10m_d@basin_test"},{"param":"stream_rast","value":"circle"},{"param":"memory","value":"1500"}],"outputs":[{"param":"basins","value":"point_basin_cloud"}]},{"module":"r.to.vect","id":"r.to.vect_1804289383","inputs":[{"param":"input","value":"point_basin_cloud"},{"param":"type","value":"area"},{"param":"column","value":"value"}],"outputs":[{"

## Expot to GCS

In [59]:

!curl -u 'actinia-gdi:actinia-gdi' -X POST "http://localhost:8088/api/v3/locations/nc_spm_08/processing_async_export_gcs" -H "Content-Type: application/json" \
   -d $PC_BODY | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    46  100    37  100     9    129     31 --:--:-- --:--:-- --:--:--   161
{
  "message": "Internal Server Error"
}
curl: (3) bad range in URL position 7:
list:[module:g.region
      ^


In [111]:
url = "http://localhost:8088/api/v3/locations/nc_spm_08/processing_async_export_gcs"
# url = "http://localhost:8088/api/v3/locations/nc_spm_08/processing_export"

pc = !ace location="nc_spm_08" mapset="basin_test" script="./scripts/basin.sh" -d | jq -r -c
print(pc)
PC_BODY = json.loads(pc[0])
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=PC_BODY,
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

['{"version":"1","list":[{"module":"g.region","id":"g.region_1804289383","inputs":[{"param":"n","value":"252702.4094174725"},{"param":"s","value":"229056.70015573464"},{"param":"e","value":"618028.8126522538"},{"param":"w","value":"598579.6858165322"},{"param":"align","value":"direction_3k_10m_d@basin_test"}]},{"module":"r.circle","id":"r.circle_1804289383","flags":"b","inputs":[{"param":"coordinates","value":"601225.5895524962,241014.7657618352"},{"param":"max","value":"200"}],"outputs":[{"param":"output","value":"circle"}]},{"module":"r.stream.basins","id":"r.stream.basins_1804289383","flags":"c","inputs":[{"param":"direction","value":"direction_3k_10m_d@basin_test"},{"param":"stream_rast","value":"circle"},{"param":"memory","value":"1500"}],"outputs":[{"param":"basins","value":"point_basin_cloud"}]},{"module":"r.to.vect","id":"r.to.vect_1804289383","inputs":[{"param":"input","value":"point_basin_cloud"},{"param":"type","value":"area"},{"param":"column","value":"value"}],"outputs":[{

In [153]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8080  100  8080    0     0  25250      0 --:--:-- --:--:-- --:--:-- 25250
{
  "accept_datetime": "2022-05-03 03:19:14.362270",
  "accept_timestamp": 1651547954.3622699,
  "api_info": {
    "endpoint": "asyncephemeralexportgcsresource",
    "method": "POST",
    "path": "/api/v3/locations/nc_spm_08/processing_async_export_gcs",
    "request_url": "http://localhost:8088/api/v3/locations/nc_spm_08/processing_async_export_gcs"
  },
  "datetime": "2022-05-03 09:51:26.023481",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "id": "g.region_1804289383",
          "inputs": [
            {
              "param": "n",
              "value": "252702.4094174725"
            },
            {
              "param": "s",
              "value": "229056.

In [148]:
# GRASS GeoServer

In [106]:
# GRASS MBTiles

In [ ]:
# PostGIS VectorTiles

In [99]:
!importer raster=nlcd_2019@https://storage.googleapis.com/tomorrownow-actinia-dev/nlcd/nlcd_2019_cog.tif

In [ ]:
resource_id-2af75017-eae1-4051-8650-bb83627ec19d

In [158]:
# !ace --help
!ace info_job=resource_id-2af75017-eae1-4051-8650-bb83627ec19d
# !ace kill_job=resource_id-2af75017-eae1-4051-8650-bb83627ec19d

{'accept_datetime': '2022-05-03 16:51:46.637922',
 'accept_timestamp': 1651596706.6379216,
 'api_info': {'endpoint': 'asyncpersistentresource',
              'method': 'POST',
              'path': '/api/v3/locations/nc_spm_08/mapsets/basin_test/processing_async',
              'request_url': 'http://actinia-core:8088/api/v3/locations/nc_spm_08/mapsets/basin_test/processing_async'},
 'datetime': '2022-05-03 17:00:02.620220',
 'http_code': 200,
 'message': 'AsyncProcessTermination:  Process <r.import> was terminated by '
            'user request',
 'process_chain_list': [{'list': [{'id': 'g.region_1804289',
                                   'inputs': [{'param': 'align',
                                               'value': 'direction_3k_10m_d'},
                                              {'param': 'n',
                                               'value': '243469.90223777824'},
                                              {'param': 'e',
                                        

In [3]:
!ace list_jobs="running"
# !ace info_job=resource_id-2af75017-eae1-4051-8650-bb83627ec19d
# !ace kill_job=resource_id-2af75017-eae1-4051-8650-bb83627ec19d

resource_id-4ac0a08e-5e80-4020-9174-c2ed8b500d4e running 2022-05-12 22:17:16.328837


In [4]:
!ace info_job=resource_id-4ac0a08e-5e80-4020-9174-c2ed8b500d4e

{'accept_datetime': '2022-05-12 21:58:57.603388',
 'accept_timestamp': 1652392737.6033869,
 'api_info': {'endpoint': 'asyncpersistentresource',
              'method': 'POST',
              'path': '/api/v3/locations/nc_spm_08/mapsets/basin_test/processing_async',
              'request_url': 'http://actinia-core:8088/api/v3/locations/nc_spm_08/mapsets/basin_test/processing_async'},
 'datetime': '2022-05-12 22:17:36.404979',
 'http_code': 200,
 'message': 'Running executable r.import with parameters '
            "['input=/vsicurl/https://prd-tnm.s3.amazonaws.com/ ... 00', "
            "'extent=region', 'output=usgs_3dep_30m', '-'] for 1118.52 seconds",
 'process_chain_list': [],
 'progress': {'num_of_steps': 25, 'step': 2},
 'resource_id': 'resource_id-4ac0a08e-5e80-4020-9174-c2ed8b500d4e',
 'status': 'running',
 'time_delta': 1118.8016052246094,
 'timestamp': 1652393856.404978,
 'urls': {'resources': [],
          'status': 'http://actinia-core:8088/api/v3/resources/actinia-gdi/reso

In [5]:
!ace kill_job=resource_id-4ac0a08e-5e80-4020-9174-c2ed8b500d4e

{'message': 'Termination request committed', 'status': 'accepted'}


In [2]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/CONUS/mapsets/hydro/lock" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   793  100   793    0     0   1341      0 --:--:-- --:--:-- --:--:--  1341
{
  "accept_datetime": "2022-05-18 20:27:33.523593",
  "accept_timestamp": 1652905653.5235925,
  "api_info": {
    "endpoint": "mapsetlockmanagementresource",
    "method": "GET",
    "path": "/api/v3/locations/CONUS/mapsets/hydro/lock",
    "request_url": "http://localhost:8088/api/v3/locations/CONUS/mapsets/hydro/lock"
  },
  "datetime": "2022-05-18 20:27:33.535362",
  "http_code": 200,
  "message": "Mapset lock state: True",
  "process_chain_list": [],
  "process_log": [],
  "process_results": true,
  "progress": {
    "num_of_steps": 0,
    "step": 0
  },
  "resource_id": "resource_id-8e07dca9-acf9-4bff-b46d-15e029dbd7aa",
  "status": "finished",
  "time_delta": 0.011802196502685547,
  "timestamp": 1652905653.5353475,
  "urls": {
    "resources": []

In [1]:
!curl -u 'actinia-gdi:actinia-gdi' -X DELETE "http://localhost:8088/api/v3/locations/CONUS/mapsets/hydro/lock" | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   807  100   807    0     0   1386      0 --:--:-- --:--:-- --:--:--  1386
{
  "accept_datetime": "2022-05-20 04:19:05.066118",
  "accept_timestamp": 1653020345.0661159,
  "api_info": {
    "endpoint": "mapsetlockmanagementresource",
    "method": "DELETE",
    "path": "/api/v3/locations/CONUS/mapsets/hydro/lock",
    "request_url": "http://localhost:8088/api/v3/locations/CONUS/mapsets/hydro/lock"
  },
  "datetime": "2022-05-20 04:19:05.086544",
  "http_code": 200,
  "message": "Mapset <hydro> successfully unlocked",
  "process_chain_list": [],
  "process_log": [],
  "process_results": {},
  "progress": {
    "num_of_steps": 0,
    "step": 0
  },
  "resource_id": "resource_id-bc1da67c-b4ae-49d6-afc5-a43e25b46856",
  "status": "finished",
  "time_delta": 0.020448923110961914,
  "timestamp": 1653020345.0865235,
  "urls": {
    "